In [3]:
import pandas as pd

In [51]:
read_file = pd.read_excel ("MSN Codes.xlsx")
read_file.to_csv ("MSN Codes.csv", index = None, header=True)
read_file = pd.read_excel ("Investment_Data_Train.xlsx")
read_file.to_csv ("Investment_Data_Train.csv", index = None, header=True)
# read_file = pd.read_excel ("Investment_Data_2020.xlsx")
# read_file.to_csv ("Investment_Data_2020.csv", index = None, header=True)

In [20]:
msn_code = pd.read_csv('MSN Codes.csv')

investment_data = pd.read_csv("Investment_Data_Train.csv")

In [52]:
msn_code.head()

,MSN,Description,Unit
0,BDFDB,Biomass inputs (feedstock) to the production o...,Billion Btu
1,BDPRP,Biodiesel production,Thousand barrels
2,BFFDB,Biomass inputs (feedstock) to the production o...,Billion Btu
3,BFPRP,Biofuels production,Thousand barrels
4,CLPRB,Coal production,Billion Btu


In [53]:
investment_data.head()

,Unnamed: 0,MSN,StateCode,Year,Amount,State,CO2 Emissions (Mmt),TotalNumberofInvestments,TotalAmountofAssistance
0,0,BDFDB,AK,2015,21.0,Alaska,35.027804,16.0,3345612.0
1,1,BDPRP,AK,2015,4.0,Alaska,35.027804,16.0,3345612.0
2,2,BFFDB,AK,2015,21.0,Alaska,35.027804,16.0,3345612.0
3,3,BFPRP,AK,2015,4.0,Alaska,35.027804,16.0,3345612.0
4,4,CLPRB,AK,2015,17747.0,Alaska,35.027804,16.0,3345612.0


In [25]:
investment_data.columns

Index(['Unnamed: 0', 'MSN', 'StateCode', 'Year', 'Amount', 'State',
       'CO2 Emissions (Mmt)', 'TotalNumberofInvestments',
       'TotalAmountofAssistance'],
      dtype='object')

In [54]:
single_variables = investment_data[[
    'StateCode', 'Year', 'State',
    'CO2 Emissions (Mmt)', 'TotalNumberofInvestments', 'TotalAmountofAssistance'
]].drop_duplicates()
single_variables

,StateCode,Year,State,CO2 Emissions (Mmt),TotalNumberofInvestments,TotalAmountofAssistance
0,AK,2015,Alaska,35.027804,16.0,3345612.0
29,AL,2015,Alabama,119.200889,164.0,1864412.0
58,AR,2015,Arkansas,59.053365,477.0,13453887.0
87,AZ,2015,Arizona,94.978784,65.0,1724853.0
116,CA,2015,California,351.408516,1023.0,30077512.0
...,...,...,...,...,...,...
7497,WI,2019,Wisconsin,94.822424,103.0,2915140.0
7526,WV,2019,West Virginia,85.445996,44.0,790383.0
7555,WY,2019,Wyoming,59.096649,32.0,570570.0
7584,X3,2019,Federal Offshore - Gulf of Mexico,NaN,NaN,NaN


In [56]:
multi_variables = investment_data[[
    'StateCode', 'Year', 'State',
    'MSN', 'Amount'
]].drop_duplicates()

multi_variables

,StateCode,Year,State,MSN,Amount
0,AK,2015,Alaska,BDFDB,21.000
1,AK,2015,Alaska,BDPRP,4.000
2,AK,2015,Alaska,BFFDB,21.000
3,AK,2015,Alaska,BFPRP,4.000
4,AK,2015,Alaska,CLPRB,17747.000
...,...,...,...,...,...
7590,X3,2019,Federal Offshore - Gulf of Mexico,TEPRB,5106671.000
7591,X5,2019,Federal Offshore - Pacific,COPRK,5.698
7592,X5,2019,Federal Offshore - Pacific,PAPRB,25350.000
7593,X5,2019,Federal Offshore - Pacific,PAPRP,4449.000


In [60]:
multi_variables.pivot(index=['StateCode', 'Year', 'State'], columns='MSN', values='Amount')

MSN                                        BDFDB  BDPRP  BFFDB  BFPRP  \
StateCode Year State                                                    
AK        2015 Alaska                       21.0    4.0   21.0    4.0   
          2016 Alaska                       27.0    5.0   27.0    5.0   
          2017 Alaska                       29.0    5.0   29.0    5.0   
          2018 Alaska                       15.0    3.0   15.0    3.0   
          2019 Alaska                        0.0    0.0    0.0    0.0   
...                                          ...    ...    ...    ...   
X5        2015 Federal Offshore - Pacific    NaN    NaN    NaN    NaN   
          2016 Federal Offshore - Pacific    NaN    NaN    NaN    NaN   
          2017 Federal Offshore - Pacific    NaN    NaN    NaN    NaN   
          2018 Federal Offshore - Pacific    NaN    NaN    NaN    NaN   
          2019 Federal Offshore - Pacific    NaN    NaN    NaN    NaN   

MSN                                          CLPRB   CLPRK   CLPRP  COPRK  \
StateCode Year State                                                        
AK        2015 Alaska                      17747.0  15.073  1177.0  5.717   
          2016 Alaska                      13942.0  14.957   932.0  5.722   
          2017 Alaska                      14365.0  14.978   959.0  5.723   
          2018 Alaska                      13752.0  15.253   902.0  5.706   
          2019 Alaska                      14867.0  15.252   975.0  5.698   
...                                            ...     ...     ...    ...   
X5        2015 Federal Offshore - Pacific      NaN     NaN     NaN  5.717   
          2016 Federal Offshore - Pacific      NaN     NaN     NaN  5.722   
          2017 Federal Offshore - Pacific      NaN     NaN     NaN  5.723   
          2018 Federal Offshore - Pacific      NaN     NaN     NaN  5.706   
          2019 Federal Offshore - Pacific      NaN     NaN     NaN  5.698   

MSN                                        EMFDB  ENPRP  ...    REPRB  SOTCB  \
StateCode Year State                                     ...                   
AK        2015 Alaska                        0.0    0.0  ...  23807.0    3.0   
          2016 Alaska                        0.0    0.0  ...  25155.0   14.0   
          2017 Alaska                        0.0    0.0  ...  23620.0   20.0   
          2018 Alaska                        0.0    0.0  ...  24158.0   28.0   
          2019 Alaska                        0.0    0.0  ...  22754.0   40.0   
...                                          ...    ...  ...      ...    ...   
X5        2015 Federal Offshore - Pacific    NaN    NaN  ...      NaN    NaN   
          2016 Federal Offshore - Pacific    NaN    NaN  ...      NaN    NaN   
          2017 Federal Offshore - Pacific    NaN    NaN  ...      NaN    NaN   
          2018 Federal Offshore - Pacific    NaN    NaN  ...      NaN    NaN   
          2019 Federal Offshore - Pacific    NaN    NaN  ...      NaN    NaN   

MSN                                            TEPRB     TETCB  WDEXB   WDPRB  \
StateCode Year State                                                            
AK        2015 Alaska                      1429942.0  614242.0    0.0  6865.0   
          2016 Alaska                      1435363.0  593662.0    0.0  7636.0   
          2017 Alaska                      1454196.0  601492.0    0.0  6501.0   
          2018 Alaska                      1410604.0  604644.0    0.0  6910.0   
          2019 Alaska                      1370342.0  605689.0    0.0  6418.0   
...                                              ...       ...    ...     ...   
X5        2015 Federal Offshore - Pacific    65465.0       NaN    NaN     NaN   
          2016 Federal Offshore - Pacific    35156.0       NaN    NaN     NaN   
          2017 Federal Offshore - Pacific    32701.0       NaN    NaN     NaN   
          2018 Federal Offshore - Pacific    27811.0       NaN    NaN     NaN   
          2019 Federal Offshore - P

In [47]:
LR_df = investment_data[['StateCode', 'Year', 'TotalAmountofAssistance']].drop_duplicates()
LR_df

,StateCode,Year,TotalAmountofAssistance
0,AK,2015,3345612.0
29,AL,2015,1864412.0
58,AR,2015,13453887.0
87,AZ,2015,1724853.0
116,CA,2015,30077512.0
...,...,...,...
7497,WI,2019,2915140.0
7526,WV,2019,790383.0
7555,WY,2019,570570.0
7584,X3,2019,NaN
